In [11]:
import subprocess
import glob

In [12]:
speechs = glob.glob("speech/*.wav")
speechs.sort()

In [ ]:
bin = "build/bin/whisper-cli"
model = "models/ggml-medium.bin"
model = "models/ggml-tiny-q8_0.bin"
model = "models/ggml-tiny.bin"
model = "models/ggml-small.bin"

cmds = [
  bin,
  "--model", model,
  "-l", "zh",
  "-t", "8",
  # "-ng"
]
cmds += speechs

ret = subprocess.run(cmds)

In [64]:
# load wav file
import wave
import numpy as np

def load_wav(filename):
  with wave.open(filename, "rb") as f:
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    assert nchannels == 1
    assert sampwidth == 2
    assert framerate == 16000
    data = f.readframes(nframes)
    data = np.frombuffer(data, dtype=np.int16)
    return data

def get_amplitude(data):
  amplitude = []
  last_amplitude = 0
  for i in range(len(data)):
    if data[i] > last_amplitude:
      last_amplitude = data[i]
    else:
      last_amplitude = last_amplitude * 0.999 + data[i] * 0.001
    amplitude.append(last_amplitude)

  return np.array(amplitude)

def get_speech_segment(in_speech):
  speech_segment = []
  current_in_speech = False
  not_in_speech_count = 0
  for i in range(len(in_speech)):
    if in_speech[i]:
      not_in_speech_count = 0
      current_in_speech = True
    else:
      not_in_speech_count += 1
      current_in_speech = False
    if current_in_speech or not_in_speech_count < silent_duration_threshold:
      speech_segment.append(1)
    else:
      speech_segment.append(0)
  return np.array(speech_segment)

sample_rate = 16000
noise_threshold = 0.05
silent_duration_threshold = 1.0 * sample_rate


def draw_speech_segment(wav_file):
  data = load_wav(wav_file)
  data = data.astype(np.float32) / 32768.0


  abs_data = np.abs(data)

  abs_data = get_amplitude(abs_data)

  in_speech = np.where(abs_data > noise_threshold, 1, 0)


  %matplotlib tk

  import matplotlib.pyplot as plt

  # draw a horizontal line of y = noise_threshold
  plt.axhline(noise_threshold, color = 'mediumseagreen')
  plt.plot(data)
  plt.plot(abs_data)
  plt.plot(in_speech)
  plt.plot(get_speech_segment(in_speech))


draw_speech_segment(speechs[15])

